如果是在一个金属表面吸附一个原子，一个原子在表面上有两个自由度，在金属表面上构成一个能量曲面。
如果是在表面吸附一个多原子分子，由于每个原子都有自由度，将会有更高维度的能量曲面。
我们的目标是通过训练模型及其能量，知悉整个高维能量曲面。

In [1]:
from ase.build import bulk
from ase.optimize import LBFGS
from ase.calculators.emt import EMT
from ase.visualize import view
from ase.io import write
from ase.build import fcc100, add_adsorbate, molecule, fcc111
from ase.constraints import FixAtoms, FixCartesian
from ase.atoms import Atoms
import os
import numpy as np


Function part

In [2]:
def compare_atoms(ini_atoms, pre_atoms):
    # 函数用于计算应变，变形前模型：ini_atoms, 变形后模型：pre_atoms，两个模型的属性是 ase.atoms.Atoms，
    # 如果两个模型不是Atoms(Type Error)，或者不具有应变变换(Value Error)，会提示错误。

    isAtoms = isinstance(ini_atoms, Atoms) + isinstance(pre_atoms, Atoms)
    len_queal= len(ini_atoms.positions) == len(pre_atoms.positions)
    if isAtoms*len_queal == 0:
        print('Two model are Atoms:', isAtoms==2)
        print('Two models with equal atomic numbers :', len_queal==1)
        raise TypeError("Model should be Atoms")
    
    ini_cor = np.concatenate((ini_atoms.cell.array, ini_atoms.positions), axis=0)  # 盒子和原子都统一至坐标 
    pre_cor = np.concatenate((pre_atoms.cell.array, pre_atoms.positions), axis=0)
    np.seterr(divide='ignore', invalid='ignore')  # 忽略警告，因为坐标值有0值做除数
    strains = pre_cor / ini_cor  # 坐标相除，获得比例

    strains = np.array(strains) # 转换为 numpy 数组，以便使用 numpy 的函数
    max_values = np.nanmax(strains, axis=0) # 按列计算最大值，忽略 NaN 值
    nan_mask = np.isnan(strains) # 找出所有的 NaN 值
    strains[nan_mask] = np.take(max_values, np.where(nan_mask)[1]) # 替换 NaN 值为对应列的最大值
    strains.tolist() # 转换回 Python 列表并返回

    strain_list = np.full(3, np.nan)
    for column in range(3):  # 检查每一列是否都具有相同的应变
        strains_column = strains[:,column]
        strain_equal = all(x == strains_column[0] for x in strains_column)
        if strain_equal ==1:
            strain_list[column] = strains_column[0]
        else:
            raise ValueError("Strain are not uniform")
    strain_list = strain_list-1
    print('strain: ',strain_list )
    return(strain_list)

In [3]:
def cal_LBFGC(ini_model, potential = EMT, fmax=1E-6, steps=1000):
    # 执行动力学过程，默认的势函数是EMT，力收敛判断值1E-6，最大动力学步数1E3
    ini_model.set_calculator(potential()) # setting the calculated potential 
    # 创建 LBFGS 实例
    dyn = LBFGS(ini_model)
    # 进行能量最小化优化计算
    dyn.run(fmax, steps)

    # 输出优化后的结构信息和能量值
    opt_config = dyn.atoms # initial model
    opt_energy = dyn.atoms.get_potential_energy()
    print('Limited-memory Broyden-Fletcher-Goldfarb-Shanno algorithm')
    return(opt_config, opt_energy)

In [4]:
def set_strain(ini_model, strain=[0,0,0], is_opt=True):
    # strain 表面应变由三个值控制 [ε1 ε2 ε6]
    isAtoms = isinstance(ini_model, Atoms)
    if isAtoms == 0:
        raise TypeError("Model should be Atoms")
    
    strain_slab = ini_model.copy()
    # 获取当前的晶格矩阵, 复制初始无应变构型
    cell = strain_slab.get_cell()

    strains = np.array([[strain[0], strain[2]],
               [strain[2], strain[1]]])
    deform = strains + np.identity(2)
    cell[:2,:2] = np.dot(cell[:2,:2], deform) 
    # 将新的晶格矩阵赋值给 原始 Cu 对象 
    strain_slab.set_cell(cell, scale_atoms=True, apply_constraint=False)  # scale_Atoms=True must be set to True to ensure that ...
    
    # ...the atomic coordinates adapt to changes in the lattice matrix
    
    if is_opt==True:
        opt_strain_slab, opt_strain_energr = cal_LBFGC(strain_slab)
    else:
        opt_strain_slab, opt_strain_energr = strain_slab, False
    # strain = compare_atoms(ini_model,strain_slab)
    return(opt_strain_slab, opt_strain_energr)

1. initial cofiguration of Cu bulk

In [5]:
adslab = fcc111("Cu", size=(3, 3, 3),periodic=True)
# cons= FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
# adslab.set_constraint(cons)
# adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)
# adslab.set_calculator(EMT()) # setting the calculated potential 
# print(adslab.positions[:,0])
# print(type(adslab))
opt_config, opt_energe= cal_LBFGC(adslab)
print('\n inipotential = ', opt_energe, '\n')
strain = compare_atoms(adslab,adslab)


       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 13:44:39       -0.153401*       0.0000
Limited-memory Broyden-Fletcher-Goldfarb-Shanno algorithm

 inipotential =  -0.15340080668105394 

strain:  [0. 0. 0.]


In [6]:
print(opt_config.cell)
strain_model, strain_energe= set_strain(opt_config, strain=[0.1,0,0],is_opt=True)
strain = compare_atoms(adslab,strain_model)

Cell([[7.65796644025031, 0.0, 0.0], [3.828983220125155, 6.6319934785854535, 0.0], [0.0, 0.0, 6.252703415323648]])
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 13:44:39        1.780340*       0.0000
Limited-memory Broyden-Fletcher-Goldfarb-Shanno algorithm
strain:  [0.1 0.  0. ]


In [7]:
from ase.io import read
from ase.calculators.emt import EMT
from ase.optimize import BFGS

def optimize_structure(atoms):
    # 读取初始结构信息
    # 创建势函数对象
    calc = EMT()
    atoms.set_calculator(calc)

    # 进行 BFGS 优化
    dyn = LBFGS(atoms)
    dyn.run(fmax=0, steps=100)

    # 打印优化前后的晶格参数
    print('Initial cell parameters: ', atoms.cell)
    print('Final cell parameters: ', atoms.get_cell())

    # 返回优化后的结构信息
    return atoms

# 调用函数进行结构优化
optimized_structure = optimize_structure(strain_model)


       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 13:44:39        1.780340*       0.0000
LBFGS:    1 13:44:39        1.780340*       0.0000
LBFGS:    2 13:44:39       94.770000*       0.0000
LBFGS:    3 13:44:39       94.770000*       0.0000
LBFGS:    4 13:44:39       94.770000*       0.0000
LBFGS:    5 13:44:39       94.770000*       0.0000
LBFGS:    6 13:44:39       94.770000*       0.0000
LBFGS:    7 13:44:39       94.770000*       0.0000
LBFGS:    8 13:44:39       94.770000*       0.0000
LBFGS:    9 13:44:39       94.770000*       0.0000
LBFGS:   10 13:44:39       94.770000*       0.0000
LBFGS:   11 13:44:39       94.770000*       0.0000
LBFGS:   12 13:44:39       94.770000*       0.0000
LBFGS:   13 13:44:39       94.770000*       0.0000
LBFGS:   14 13:44:39       94.770000*       0.0000
LBFGS:   15 13:44:39       94.770000*       0.0000
LBFGS:   16 13:44:39       94.770000*       0.0000
LBFGS:   17 13:44:39       94.7700

In [8]:
strain_model.constraints
help(strain_model.set_cell)

Help on method set_cell in module ase.atoms:

set_cell(cell, scale_atoms=False, apply_constraint=True) method of ase.atoms.Atoms instance
    Set unit cell vectors.
    
    Parameters:
    
    cell: 3x3 matrix or length 3 or 6 vector
        Unit cell.  A 3x3 matrix (the three unit cell vectors) or
        just three numbers for an orthorhombic cell. Another option is
        6 numbers, which describes unit cell with lengths of unit cell
        vectors and with angles between them (in degrees), in following
        order: [len(a), len(b), len(c), angle(b,c), angle(a,c),
        angle(a,b)].  First vector will lie in x-direction, second in
        xy-plane, and the third one in z-positive subspace.
    scale_atoms: bool
        Fix atomic positions or move atoms with the unit cell?
        Default behavior is to *not* move the atoms (scale_atoms=False).
    apply_constraint: bool
        Whether to apply constraints to the given cell.
    
    Examples:
    
    Two equivalent ways t

In [9]:
strain_slab = opt_config.copy()
# 获取当前的晶格矩阵, 复制初始无应变构型
strains = strain_slab.get_cell()
# 在 x 方向上添加应变
strains[:,0] *= 1.1
# 将新的晶格矩阵赋值给 原始 Cu 对象 
strain_slab.set_cell(strains, scale_atoms=True)  # scale_Atoms=True must be set to True to ensure that ...
# ...the atomic coordinates adapt to changes in the lattice matrix
strain = compare_atoms(adslab,strain_slab)
print(strain_slab.get_cell)

strain:  [0.1 0.  0. ]
<bound method Atoms.get_cell of Atoms(symbols='Cu27', pbc=True, cell=[[8.423763084275341, 0.0, 0.0], [4.211881542137671, 6.6319934785854535, 0.0], [0.0, 0.0, 6.252703415323648]], tags=...)>


In [10]:
# 创建一个 FixCartesian 约束对象，冻结 x、y 和 z 方向上的自由度
FixCartesian(strain_slab, mask=[True, True, False])
strain_slab.set_pbc(True)
strain_slab.set_calculator(EMT()) # setting the calculated potential 

In [11]:
strain_slab.set_calculator(EMT()) # setting the calculated potential 
# 创建 LBFGS 实例
dyn = LBFGS(strain_slab)
# 进行能量最小化优化计算
dyn.run(fmax=1E-8, steps=100)

# 输出优化后的结构信息和能量值
opt_config = dyn.atoms # initial model
opt_energy = dyn.atoms.get_potential_energy()
strain = compare_atoms(adslab,opt_config)

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 13:44:41        1.780340*       0.0000
strain:  [0.1 0.  0. ]


In [12]:
if __name__ == '__main__':
    # 这里是你要执行的代码
    print('start')

start
